## 1 - Importation des librairies

In [631]:
import pandas as pd 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


## 2 - Lecture de l'ensemble de donnees.

In [632]:
data_charbonnier = pd.read_excel("Données Abong-Mbang.xlsx", sheet_name="Feuil1")
data_charbonnier.head(2)

,Identifiant,ID et NOM DE L'ENQUETE,SEXE,AGE,VILLAGE DE RESIDENCE,NIVEAU D'ETUDE,ORIGINE,ETHNIE,STATUT MATRIMONIAL,PROFESSION,...,MOYEN D'ACQUISITION DES REBUTS,MOYEN DE VENTE DE CHARBON,BENEFICES TIRES DE LA CARBONISATION DES REBUTS,AVEZ-VOUS ÉTÉ FORME PAR QUELLE ORGANISATION,AVNTAGES DE LA CARBONISATION DES REBUTS DE SCIERIES,CE QUI EST PLUS BENEFIQUE ENTRE LA COUPE D'ARBRE ET LES REBUTS DANS LA CAEBONISATION,CE QUI EST MOINS PENIBLE ENTRE LA COUPE D'ARBRE ET LES REBUTS DANS LA CAEBONISATION,CE QUI DONNE PLUS DE CHARBON ENTRE LA COUPE D'ARBRE ET LES REBUTS DANS LA CAEBONISATION,CONTRAINTES LIEES A LA CARBONISATION DES REBUTS,RAISON D'INSTALLATION A COTE DES SCIERIES
0,P1S1,P1S1 Alain serge bertrang,masculin,15-30,Abong-Mbang II,secondaire,Ouest,Bamiléké,célibataire,producteur de charbon,...,achat,charbon précommandé,quantité et choix de qualité,non,deforestation controlée,rebuts de bois,rebuts de bois,rebuts de bois,mauvaise fois,diminution de la manutension
1,P2S2,P2S2 MENPUOLIA FLORE,masculin,15-30,Abong-Mbang II,secondaire,Est,maka,célibataire,producteur de charbon,...,achat,sur le marché,diminue la coupe,non,facile a carconiser,rebuts de bois,rebuts de bois,rebuts de bois,manque de rebuts,gagnner beaucoup


## 3 - Netoyage des donnees 

In [633]:
# taille de l'ensemble de donnees 
data_charbonnier.shape

(99, 33)

In [634]:
# type de donnees
data_charbonnier.describe()

,PRODUCTION MENSUELLE,NOMBRE DE MEUL MENSUELLE,NOMBRE DE SACS PAR MEULE
count,99.000000,98.000000,99.000000
mean,197.171717,4.918367,35.353535
std,489.765751,5.940727,30.262900
min,15.000000,1.000000,2.000000
25%,50.000000,2.000000,20.000000
50%,100.000000,4.000000,25.000000
75%,200.000000,5.000000,42.500000
max,4800.000000,50.000000,240.000000


In [635]:
# verication des valeurs manquantes
for col in list(data_charbonnier.columns):
    if data_charbonnier[col].isna().any()==True:
        print(col, data_charbonnier[col].isna().sum())

AGE 1
AUTRES ACTIVITES 80
FACTEURS INFLENCANT LA PRODUCTION 17
QUANTITE DE REBUTS MENSUELLE 1
NOMBRE DE MEUL MENSUELLE 1
ESPECES UTILISEES 31
BUT DE LA PRODUCTION DDU CHARBON 1
OBSTACLES LIES A LACARBONISATION 4
BENEFICES TIRES DE LA CARBONISATION DES REBUTS 21
AVNTAGES DE LA CARBONISATION DES REBUTS DE SCIERIES 20
CONTRAINTES LIEES A LA CARBONISATION DES REBUTS 31
RAISON D'INSTALLATION A COTE DES SCIERIES 16


In [636]:
# traitement des valeurs manquantes: 
    # - Remplacement des valeurs manquantes sur les colonnes "AGE","QUANTITE DE REBUTS MENSUELLE","BUT DE LA PRODUCTION DDU CHARBON" et "OBSTACLES LIES A LACARBONISATION" par les valeurs les plus frequents sur chacune des colonnes.

str_cols_with_missing_data = ["AGE","QUANTITE DE REBUTS MENSUELLE","BUT DE LA PRODUCTION DDU CHARBON","OBSTACLES LIES A LACARBONISATION","NOMBRE DE MEUL MENSUELLE"]

def most_frequent(cols):
    for col in cols:
        mode = data_charbonnier[col].mode()[0]
        data_charbonnier[col] = data_charbonnier[col].fillna(mode)
        
most_frequent(str_cols_with_missing_data)

In [637]:
# reverication des valeurs manquantes
for col in list(data_charbonnier.columns):
    if data_charbonnier[col].isna().any()==True:
        print(col, data_charbonnier[col].isna().sum())

AUTRES ACTIVITES 80
FACTEURS INFLENCANT LA PRODUCTION 17
ESPECES UTILISEES 31
BENEFICES TIRES DE LA CARBONISATION DES REBUTS 21
AVNTAGES DE LA CARBONISATION DES REBUTS DE SCIERIES 20
CONTRAINTES LIEES A LA CARBONISATION DES REBUTS 31
RAISON D'INSTALLATION A COTE DES SCIERIES 16


remplacons toute les colonnes qui ont une valeur manquante superieur ou egale a 15 par non "valeure non definie"

In [638]:
data_charbonnier.fillna(value="valeure non definie",inplace=True)

In [639]:
# reverication des valeurs manquantes
for col in list(data_charbonnier.columns):
    if data_charbonnier[col].isna().any()==True:
        print(col, data_charbonnier[col].isna().sum())

nous constattons qu'il n'y a plus de valeure manquantes

In [640]:
# preoccupons nous a present des colonnes numeriques tels que: PRODUCTION MENSUELLE, NOMBRE DE MEUL MENSUELLE, NOMBRE DE SACS PAR MEULE
col_to_process = ["PRODUCTION MENSUELLE","NOMBRE DE MEUL MENSUELLE","NOMBRE DE SACS PAR MEULE"]
num_data = data_charbonnier[col_to_process].head()

In [641]:
# verifions les types 
num_data.dtypes

PRODUCTION MENSUELLE          int64
NOMBRE DE MEUL MENSUELLE    float64
NOMBRE DE SACS PAR MEULE      int64
dtype: object

In [642]:
# changeons le types de la colonne "NOMBRE DE MEUL MENSUELLE" de float en int 
data_charbonnier["NOMBRE DE MEUL MENSUELLE"] = data_charbonnier["NOMBRE DE MEUL MENSUELLE"].astype("int64")

In [643]:
# reverifions les types
num_data = data_charbonnier[col_to_process].head()
num_data.dtypes

PRODUCTION MENSUELLE        int64
NOMBRE DE MEUL MENSUELLE    int64
NOMBRE DE SACS PAR MEULE    int64
dtype: object

### Magiscule et minuscule

Nous constatons que sur la colonne "ETHNIE", il les ethnies sont en doublons ( Magiscule et Minuscule), nous devons donc les normaliser.

In [644]:
# colonne ETHNIE 
data_charbonnier.loc[data_charbonnier["ETHNIE"].isin(["Bamoun","bamoun"]),"ETHNIE"] = "Bamoun"
data_charbonnier.loc[data_charbonnier["ETHNIE"].isin(["Bana","bana"]),"ETHNIE"] = "Bana"
data_charbonnier.loc[data_charbonnier["ETHNIE"].isin(["Bassa","bassa"]),"ETHNIE"] = "Bassa"
data_charbonnier.loc[data_charbonnier["ETHNIE"].isin(["Beti","beti"]),"ETHNIE"] = "Beti"
data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"] = data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].str.replace(r'.*demande.*', 'Demande en charbon', regex=True)
data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"] = data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].str.replace(r'.*danande.*', 'Demande en charbon', regex=True)
data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"] = data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].str.replace(r'.*démande*.', 'Demande en charbon', regex=True)
data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"] = data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].str.replace(r'.*bois*.', 'Disponibilite du bois', regex=True)

# colonne PROFESSION
data_charbonnier.loc[data_charbonnier["PROFESSION"].isin(["commerçant","commerçante"]),"PROFESSION"] = "commerçant(e)"
data_charbonnier.loc[data_charbonnier["PROFESSION"].isin(["étudiant","étudiante"]),"PROFESSION"] = "étudiant(e)"

In [645]:
data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].unique()

array(['approvisionnement', 'Demande en charbon',
       'production des scieries', 'selon le marché',
       'des rebuts sur piesd', 'des rebuts present sur place',
       "quantité d'achat", 'moyen financière',
       'la disponibilité des rebuts', 'Disponibilite du bois',
       "l'acquisition des rebuts", 'Disponibilite du boissur place',
       'rebuts disponibles', 'disponibilité des rebuts',
       'Disponibilite du boisdisponible sur place',
       'Disponibilite du boisdisponible', 'de la clientelle',
       'acquisition des rebuts', 'capacité de production',
       'de la matière première', 'du temps de production',
       'Disponibilité des rebuts', 'valeure non definie',
       'Disponibilite du boisdes scierie', 'abondance des rebuts',
       'de ce que la scierie libère', 'de la force de production',
       'de la frequence des rebuts', 'du type de four',
       'Disponibilite du boisreçut des scieries',
       "de l'approvisionnement", 'acquisition en rebuts',
       'qu

## 3 - Analyse 

fonction de repartion.

In [646]:
repartition = data_charbonnier["PROFESSION"].value_counts(normalize=True)
repartition= repartition.to_frame().reset_index()
repartition

,PROFESSION,proportion
0,producteur de charbon,0.787879
1,commerçant(e),0.101010
2,élève,0.040404
3,étudiant(e),0.040404
4,maire,0.010101
5,technicien,0.010101
6,mecanicien,0.010101


In [647]:
repartition.columns

Index(['PROFESSION', 'proportion'], dtype='object')

In [648]:
def pie_diagramme(data, col):
    # Repartition de valeurs de col en de proportion ( en pourcentage)
    repartition = data[col].value_counts(normalize=True)
    repartition= repartition.to_frame().reset_index()

    # Construction du graphique
    fig_bar = px.pie(data_frame=repartition,
                     values="proportion",
                     names=col,
                     color=col, 
                     title=f"Répartition en pourcentage des charbonniers par {col}",
                )
    
    # personnalisation du graphique 
    fig_bar.update_traces(
        textposition='inside', 
        textinfo='percent+label',
        textfont_size=20,
        marker=dict(line=dict(color='#000000', width=2)),
    )
    
    # Affichage du graphique 
    fig_bar.show()

### 1 - Répartition de charbonniers par sexe %

In [649]:
pie_diagramme(data=data_charbonnier, col="SEXE")

### 2 - Répartition des charbonniers selon leur statut matrimoniale

In [650]:
pie_diagramme(data=data_charbonnier, col="STATUT MATRIMONIAL")

### 3 - Répartition des charbonniers selon leur profession respective

In [651]:
pie_diagramme(data=data_charbonnier, col="PROFESSION")

### 4 - Répartition des charbonniers selon leur groupe ethnique.

In [652]:
pie_diagramme(data=data_charbonnier,col="ETHNIE")

### 5 - Répartition des charbonniers leur région respective

In [653]:
pie_diagramme(data=data_charbonnier,col="ORIGINE")

### 6 - Répartition des charbonniers selon leur niveau d’étude

In [654]:
pie_diagramme(data=data_charbonnier,col="NIVEAU D'ETUDE")

### 7 - Diagramme des revenues mensuelles des charbonniers ( en intervalle)

In [655]:
pie_diagramme(data=data_charbonnier,col="REVENUE MENSUELLE")

In [656]:
data_charbonnier.columns

Index(['Identifiant', 'ID et NOM DE L'ENQUETE', 'SEXE', 'AGE',
       'VILLAGE DE RESIDENCE', 'NIVEAU D'ETUDE', 'ORIGINE', 'ETHNIE',
       'STATUT MATRIMONIAL', 'PROFESSION', 'AUTRES ACTIVITES',
       'REVENUE MENSUELLE', 'DEBUT DE CAEBONISTION DES REBUTS',
       'ELEMENTENTS AYANT FACILITE LA CARBONISATION DES REBUTS',
       'PRODUCTION MENSUELLE', 'FACTEURS INFLENCANT LA PRODUCTION',
       'QUANTITE DE REBUTS MENSUELLE', 'NOMBRE DE MEUL MENSUELLE',
       'NOMBRE DE SACS PAR MEULE', 'ESPECES UTILISEES',
       'BUT DE LA PRODUCTION DDU CHARBON', 'POINTS D'ACHEMINEMENT',
       'OBSTACLES LIES A LACARBONISATION', 'MOYEN D'ACQUISITION DES REBUTS',
       'MOYEN DE VENTE DE CHARBON',
       'BENEFICES TIRES DE LA CARBONISATION DES REBUTS',
       'AVEZ-VOUS ÉTÉ FORME PAR QUELLE ORGANISATION',
       'AVNTAGES DE LA CARBONISATION DES REBUTS DE SCIERIES',
       'CE QUI EST PLUS BENEFIQUE ENTRE LA COUPE D'ARBRE ET LES REBUTS DANS LA CAEBONISATION',
       'CE QUI EST MOINS PENIBLE EN

In [657]:
pie_diagramme(data=data_charbonnier,col="DEBUT DE CAEBONISTION DES REBUTS")

### 8.	Répartition selon la durée de carbonisation de rebuts.

In [658]:
repartition_debuts_carb = data_charbonnier["DEBUT DE CAEBONISTION DES REBUTS"].value_counts(normalize=True)
repartition_debuts_carb = repartition_debuts_carb.to_frame().reset_index()
repartition_debuts_carb

,DEBUT DE CAEBONISTION DES REBUTS,proportion
0,moins de 1an,0.888889
1,1à2 ans,0.111111


In [659]:
fig = px.funnel(repartition_debuts_carb, 
                x='DEBUT DE CAEBONISTION DES REBUTS',
                y='proportion')
fig.update_traces(
    opacity=0.8,
    textposition='inside', 
    textinfo='percent total+label',
    textfont=dict(size=25, color='white'))

fig.update_layout(
    title_text="Entonnoir de Conversion",
    margin=dict(l=100, r=75, b=75, t=100),
    # paper_bgcolor='lightgrey'
)
fig.show()

### 9.	Productivité du charbon des rebuts de scieries en fonction des charbonniers formés (diagramme) (représentation de la production(quantité) du charbon en fonction de la formation reçu par les producteurs)

In [660]:
repartition_debuts_carb = data_charbonnier[["AVEZ-VOUS ÉTÉ FORME PAR QUELLE ORGANISATION"]].value_counts()
repartition_debuts_carb = repartition_debuts_carb.to_frame().reset_index()
repartition_debuts_carb

,AVEZ-VOUS ÉTÉ FORME PAR QUELLE ORGANISATION,count
0,non,88
1,GIZ,11


In [661]:
fig =go.Figure(go.Sunburst(
    labels=["Forme ?", 
            "Oui par la GIZ",
            "Non, pas forme",
            "Produit Moins de 5 Tonnnes",
            "Produit de 5a10 tonnes",
            "Produit de 10a15 tonnes",
            "Produit Moins de 5Tonnnes",
            "Produit de 5a 10 tonnes",
            "Produit de 10a 15 tonnes"],
    parents=["", 
             "Forme ?", 
             "Forme ?",
             "Oui par la GIZ",
             "Oui par la GIZ",
             "Oui par la GIZ",
             "Non, pas forme",
             "Non, pas forme",
             "Non, pas forme"],
    values=[100, 11, 88, 3, 8, 0 , 72 , 15, 1],
))
fig.update_layout(
    margin=dict(t=100, l=5, r=5, b=5),
    title_text="Répartition de la production de charbon par formation",
    title_x=0.5, # Centre le titre horizontalement
    title_font=dict(size=20, family="Arial", color="#333333"),
    font=dict(size=30, color="black")
)
fig.update_traces(
    textinfo='value+label',
)


fig.show()

### 12.	Liste des facteurs dont en dépendes la production du charbon des rebuts de scieries (un tableau)

In [662]:
import plotly.graph_objects as go
import pandas as pd

# Créer un DataFrame Pandas
df = pd.DataFrame({'Liste des facteurs':list(data_charbonnier["FACTEURS INFLENCANT LA PRODUCTION"].unique()),})

# Créer un figure avec un trace de type Table
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
               line_color='darkslategray',
               fill_color='lightskyblue',
               align='center',
               font=dict(color='white', size=12)),
    cells=dict(values=df.values.T,
               align='left',
               fill_color='lightgrey',
               font=dict(size=11)))
])
fig.update_layout(
    title="Les facteurs dont en depend la production du charbon"
)

fig.show()


### 13.	Représentation du nombre de meule constituée par charbonnier mensuellement

In [663]:
fig = px.line(data_frame=data_charbonnier, x="Identifiant",y="NOMBRE DE MEUL MENSUELLE")
fig.update_layout(

)
fig.show()

### 14.	Représentation du nombre de sacs produit en fonction du nombre de meule constituée mensuellement